In [ ]:
import import_ipynb
import data
import measure
import layer
import model
import hyperparameter_selection as hs
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import umap
import softmax_layer
import classification_model
import measurements
import activation_layer
import conv_layer
import reshape_layer
import maxpooling_layer


def measure_dimensionality():
    dims = [
        (258, 144),
        (124,64),
        (64,32)
    ]
    
    data1 = data.Data()
    data1.get_data()
    
    input_size = 784
    epochs = 20
    
    kernel_size = (3,3)
    pooling_size = (2,2)
    
#     input_shape = who_knows_what

    
    for dim in dims:
        layers = [
            layer.Layer(input_size,dim[0],model.relu,model.relu_backward),
            layer.Layer(dim[0],dim[1],model.relu,model.relu_backward),
            layer.Layer(dim[1],dim[0],model.relu,model.relu_backward),
            layer.Layer(dim[0],input_size,model.sigmoid,model.sigmoid_derivative),
        ]
    
        model1 = model.AutoencoderModel(layers,0.01,epochs,50)
        model1.train(data1.X_train, data1.y_train, data1.X_val, data1.y_val) #T
        
        # Predict on test data
        y_pred = model1.predict(data1.X_test.T)
        test_accuracy = model1.get_accuracy(y_pred,data1.X_test.T)
        print("test accuracy:")
        print(test_accuracy)

        # Show graphs
        measure.plot_graph(range(epochs), model1.accuracy_history, "Accuracy")
        measure.plot_graph(range(epochs), np.array(model1.cost_history).flatten(), "Loss")
        measure.plot_graph(range(epochs), np.array(model1.cost_history_train).flatten(), "Train loss")

        plt.imshow(y_pred.T[0].reshape((28,28))*255, cmap='gray')
        plt.axis('off')
        plt.show()
        
   
def classification_modell():
    data1 = data.Data()
    data1.get_data()

    layers = [
        layer.Layer(784,144,model.relu,model.relu_backward),
        layer.Layer(144,64,model.relu,model.relu_backward),
        layer.Layer(64,144,model.relu,model.relu_backward),
        layer.Layer(144,784,model.sigmoid,model.sigmoid_derivative),
    ]
    
    epochs = 20  # 100 is good, but could be better
    model1 = model.AutoencoderModel(layers, 0.01, epochs, 100)
    model1.train(data1.X_train, data1.X_val) #T
    
    code = model1.encode(data1.X_train.T).T
    code2 = model1.encode(data1.X_val.T).T
    code3 = model1.encode(data1.X_test.T).T
    
    layers = [
        layer.Layer(64,124,model.relu,model.relu_backward),
        layer.Layer(124,64,model.relu,model.relu_backward),
        softmax_layer.SoftmaxLayer(64,10)
    ]
    
    epochs = 100
    model2 = classification_model.ClassificationModel(layers, 0.01, epochs, 100)
    model2.train(code, data1.y_train, code2, data1.y_val) #T
    
    # Predict on test data
    y_pred = model2.predict(code3.T)
    test_accuracy = model2.get_accuracy(y_pred,data1.y_test.T)
    print("test accuracy:")
    print(test_accuracy)
    
    model1.graph_hidden_layer()
    model1.umap_image(data1.X_val.T, data1.y_val)
    model1.umap_image(data1.X_test.T, data1.y_test)
    
    # Show graphs
    measure.plot_graph(range(epochs), model2.accuracy_history, "Accuracy")
    measure.plot_graph(range(epochs), np.array(model2.cost_history).flatten(), "Loss")
    measure.plot_graph(range(epochs), np.array(model2.cost_history_train).flatten(), "Train loss")
    
    for i in range(len(y_pred)):
        plt.imshow(y_pred.T[i].reshape((28,28))*255, cmap='gray')
        plt.axis('off')
        plt.show()
    

def autoencoder_model():
    data1 = data.Data()
    data1.get_data()
    
    print(data1.X_val.shape)
    
    input_size = 784
    output_size_1 = 144
    
    kernel_size = (4,4)
    pooling_size = (2,2)
    
    layers = [
        conv_layer.ConvLayer(kernel_size),
        activation_layer.ActivationLayer(model.relu, model.relu_backward),
        maxpooling_layer.MaxPoolingLayer(pooling_size),
        conv_layer.ConvLayer(kernel_size),
        activation_layer.ActivationLayer(model.relu, model.relu_backward),
        maxpooling_layer.MaxPoolingLayer(pooling_size),
        reshape_layer.ReshapeLayer((3,20,20), (1,1200)),
        layer.Layer(1200,output_size_1,model.relu,model.relu_backward),
        layer.Layer(output_size_1,input_size,model.sigmoid,model.sigmoid_derivative),
    ]
    
    org_shape = data1.X_train
#     x_train_reshaped = data1.X_train.reshape((1,40000,28, 28))
#     x_val_reshaped = data1.X_val.reshape((1,20000,28, 28))
    
#     print("reshaped shape x train")
#     print(x_train_reshaped.shape)
    
    epochs = 10
    model1 = model.AutoencoderModel(layers, 0.01, epochs, 1)
    model1.train(data1.X_train, data1.X_val) #T
    
    # Predict on test data
    y_pred = model1.predict(data1.X_test.T)
    test_accuracy = model1.get_accuracy(y_pred,data1.X_test.T)
    print("test accuracy:")
    print(test_accuracy)
    
    model1.graph_hidden_layer()
    model1.umap_image(data1.X_train.T, data1.y_train)
    model1.umap_image(data1.X_val.T, data1.y_val)
    model1.umap_image(data1.X_test.T, data1.y_test)
    
    # Show graphs
    measure.plot_graph(range(epochs), model1.accuracy_history, "Accuracy")
    measure.plot_graph(range(epochs), np.array(model1.cost_history).flatten(), "Loss")
    measure.plot_graph(range(epochs), np.array(model1.cost_history_train).flatten(), "Train loss")
    
#     plt.imshow(y_pred.T[2].reshape((28,28))*255, cmap='gray')
#     plt.axis('off')
#     plt.show()

    # Plot the original test images 
    print("Original test images:")
    for i in range(5):
        plt.subplot(1, 5, i + 1)
        plt.imshow(data1.X_test[i].reshape((28,28))*255, cmap='gray')
        plt.axis('off')

    plt.show()

    # Plot the decoded 
    print("Decoded images:")
    for i in range(5):
        plt.subplot(1, 5, i + 1)
        plt.imshow(y_pred.T[i].reshape((28,28))*255, cmap='gray')
        plt.axis('off')

    plt.show()
    

if __name__ == '__main__':
    print("program start")
    
    autoencoder_model()
#     measure_dimensionality()
#     classification_modell()
#     c()

#     measurements.measure_dimensionality()
    
    print("program end")

importing Jupyter notebook from data.ipynb
importing Jupyter notebook from measure.ipynb
importing Jupyter notebook from layer.ipynb
importing Jupyter notebook from model.ipynb
importing Jupyter notebook from hyperparameter_selection.ipynb
importing Jupyter notebook from softmax_layer.ipynb
importing Jupyter notebook from classification_model.ipynb
importing Jupyter notebook from measurements.ipynb
importing Jupyter notebook from activation_layer.ipynb
importing Jupyter notebook from conv_layer.ipynb
importing Jupyter notebook from reshape_layer.ipynb
importing Jupyter notebook from maxpooling_layer.ipynb
program start
x_train.shape: (40000, 28, 28)
y_train.shape: (40000,)
x_test.shape: (10000, 28, 28)
y_train.shape: (10000,)
data loading done
(20000, 28, 28)
epoch: 0
batch: 0
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6


<string>:7: RuntimeWarning: overflow encountered in exp


batch: 7
batch: 8
batch: 9
batch: 10
batch: 11
batch: 12
batch: 13
batch: 14
batch: 15
batch: 16
batch: 17
batch: 18
batch: 19
batch: 20
batch: 21
batch: 22
batch: 23
batch: 24
batch: 25
batch: 26
batch: 27
batch: 28
batch: 29
batch: 30
batch: 31
batch: 32
batch: 33
batch: 34
batch: 35
batch: 36
batch: 37
batch: 38
batch: 39
batch: 40
batch: 41
batch: 42
batch: 43
batch: 44
batch: 45
batch: 46
batch: 47
batch: 48
batch: 49
batch: 50
batch: 51
batch: 52
batch: 53
batch: 54
batch: 55
batch: 56
batch: 57
batch: 58
batch: 59
batch: 60
batch: 61
batch: 62
batch: 63
batch: 64
batch: 65
batch: 66
batch: 67
batch: 68
batch: 69
batch: 70
batch: 71
batch: 72
batch: 73
batch: 74
batch: 75
batch: 76
batch: 77
batch: 78
batch: 79
batch: 80
batch: 81
batch: 82
batch: 83
batch: 84
batch: 85
batch: 86
batch: 87
batch: 88
batch: 89
batch: 90
batch: 91
batch: 92
batch: 93
batch: 94
batch: 95
batch: 96
batch: 97
batch: 98
batch: 99
batch: 100
batch: 101
batch: 102
batch: 103
batch: 104
batch: 105
batch: 

C:\Users\agata\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<string>:42: RuntimeWarning: invalid value encountered in multiply
C:\Users\agata\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


batch: 177
batch: 178
batch: 179
batch: 180
batch: 181
batch: 182
batch: 183
batch: 184
batch: 185
batch: 186
batch: 187
batch: 188
batch: 189
batch: 190
batch: 191
batch: 192
batch: 193
batch: 194
batch: 195
batch: 196
batch: 197
batch: 198
batch: 199
batch: 200
batch: 201
batch: 202
batch: 203
batch: 204
batch: 205
batch: 206
batch: 207
batch: 208
batch: 209
batch: 210
batch: 211
batch: 212
batch: 213
batch: 214
batch: 215
batch: 216
batch: 217
batch: 218
batch: 219
batch: 220
batch: 221
batch: 222
batch: 223
batch: 224
batch: 225
batch: 226
batch: 227
batch: 228
batch: 229
batch: 230
batch: 231
batch: 232
batch: 233
batch: 234
batch: 235
batch: 236
batch: 237
batch: 238
batch: 239
batch: 240
batch: 241
batch: 242
batch: 243
batch: 244
batch: 245
batch: 246
batch: 247
batch: 248
batch: 249
batch: 250
batch: 251
batch: 252
batch: 253
batch: 254
batch: 255
batch: 256
batch: 257
batch: 258
batch: 259
batch: 260
batch: 261
batch: 262
batch: 263
batch: 264
batch: 265
batch: 266
batch: 267

...

batch: 7795
batch: 7796
batch: 7797
batch: 7798
batch: 7799
batch: 7800
batch: 7801
batch: 7802
batch: 7803
batch: 7804
batch: 7805
batch: 7806
batch: 7807
batch: 7808
batch: 7809
batch: 7810
batch: 7811
batch: 7812
batch: 7813
batch: 7814
batch: 7815
batch: 7816
batch: 7817
batch: 7818
batch: 7819
batch: 7820
batch: 7821
batch: 7822
batch: 7823
batch: 7824
batch: 7825
batch: 7826
batch: 7827
batch: 7828
batch: 7829
batch: 7830
batch: 7831
batch: 7832
batch: 7833
batch: 7834
batch: 7835
batch: 7836
batch: 7837
batch: 7838
batch: 7839
batch: 7840
batch: 7841
batch: 7842
batch: 7843
batch: 7844
batch: 7845
batch: 7846
batch: 7847
batch: 7848
batch: 7849
batch: 7850
batch: 7851
batch: 7852
batch: 7853
batch: 7854
batch: 7855
batch: 7856
batch: 7857
batch: 7858
batch: 7859
batch: 7860
batch: 7861
batch: 7862
batch: 7863
batch: 7864
batch: 7865
batch: 7866
batch: 7867
batch: 7868
batch: 7869
batch: 7870
batch: 7871
batch: 7872
batch: 7873
batch: 7874
batch: 7875
batch: 7876
batch: 7877
batc

batch: 8486
batch: 8487
batch: 8488
batch: 8489
batch: 8490
batch: 8491
batch: 8492
batch: 8493
batch: 8494
batch: 8495
batch: 8496
batch: 8497
batch: 8498
batch: 8499
batch: 8500
batch: 8501
batch: 8502
batch: 8503
batch: 8504
batch: 8505
batch: 8506
batch: 8507
batch: 8508
batch: 8509
batch: 8510
batch: 8511
batch: 8512
batch: 8513
batch: 8514
batch: 8515
batch: 8516
batch: 8517
batch: 8518
batch: 8519
batch: 8520
batch: 8521
batch: 8522
batch: 8523
batch: 8524
batch: 8525
batch: 8526
batch: 8527
batch: 8528
batch: 8529
batch: 8530
batch: 8531
batch: 8532
batch: 8533
batch: 8534
batch: 8535
batch: 8536
batch: 8537
batch: 8538
batch: 8539
batch: 8540
batch: 8541
batch: 8542
batch: 8543
batch: 8544
batch: 8545
batch: 8546
batch: 8547
batch: 8548
batch: 8549
batch: 8550
batch: 8551
batch: 8552
batch: 8553
batch: 8554
batch: 8555
batch: 8556
batch: 8557
batch: 8558
batch: 8559
batch: 8560
batch: 8561
batch: 8562
batch: 8563
batch: 8564
batch: 8565
batch: 8566
batch: 8567
batch: 8568
batc

batch: 9173
batch: 9174
batch: 9175
batch: 9176
batch: 9177
batch: 9178
batch: 9179
batch: 9180
batch: 9181
batch: 9182
batch: 9183
batch: 9184
batch: 9185
batch: 9186
batch: 9187
batch: 9188
batch: 9189
batch: 9190
batch: 9191
batch: 9192
batch: 9193
batch: 9194
batch: 9195
batch: 9196
batch: 9197
batch: 9198
batch: 9199
batch: 9200
batch: 9201
batch: 9202
batch: 9203
batch: 9204
batch: 9205
batch: 9206
batch: 9207
batch: 9208
batch: 9209
batch: 9210
batch: 9211
batch: 9212
batch: 9213
batch: 9214
batch: 9215
batch: 9216
batch: 9217
batch: 9218
batch: 9219
batch: 9220
batch: 9221
batch: 9222
batch: 9223
batch: 9224
batch: 9225
batch: 9226
batch: 9227
batch: 9228
batch: 9229
batch: 9230
batch: 9231
batch: 9232
batch: 9233
batch: 9234
batch: 9235
batch: 9236
batch: 9237
batch: 9238
batch: 9239
batch: 9240
batch: 9241
batch: 9242
batch: 9243
batch: 9244
batch: 9245
batch: 9246
batch: 9247
batch: 9248
batch: 9249
batch: 9250
batch: 9251
batch: 9252
batch: 9253
batch: 9254
batch: 9255
batc

batch: 9860
batch: 9861
batch: 9862
batch: 9863
batch: 9864
batch: 9865
batch: 9866
batch: 9867
batch: 9868
batch: 9869
batch: 9870
batch: 9871
batch: 9872
batch: 9873
batch: 9874
batch: 9875
batch: 9876
batch: 9877
batch: 9878
batch: 9879
batch: 9880
batch: 9881
batch: 9882
batch: 9883
batch: 9884
batch: 9885
batch: 9886
batch: 9887
batch: 9888
batch: 9889
batch: 9890
batch: 9891
batch: 9892
batch: 9893
batch: 9894
batch: 9895
batch: 9896
batch: 9897
batch: 9898
batch: 9899
batch: 9900
batch: 9901
batch: 9902
batch: 9903
batch: 9904
batch: 9905
batch: 9906
batch: 9907
batch: 9908
batch: 9909
batch: 9910
batch: 9911
batch: 9912
batch: 9913
batch: 9914
batch: 9915
batch: 9916
batch: 9917
batch: 9918
batch: 9919
batch: 9920
batch: 9921
batch: 9922
batch: 9923
batch: 9924
batch: 9925
batch: 9926
batch: 9927
batch: 9928
batch: 9929
batch: 9930
batch: 9931
batch: 9932
batch: 9933
batch: 9934
batch: 9935
batch: 9936
batch: 9937
batch: 9938
batch: 9939
batch: 9940
batch: 9941
batch: 9942
batc

batch: 10503
batch: 10504
batch: 10505
batch: 10506
batch: 10507
batch: 10508
batch: 10509
batch: 10510
batch: 10511
batch: 10512
batch: 10513
batch: 10514
batch: 10515
batch: 10516
batch: 10517
batch: 10518
batch: 10519
batch: 10520
batch: 10521
batch: 10522
batch: 10523
batch: 10524
batch: 10525
batch: 10526
batch: 10527
batch: 10528
batch: 10529
batch: 10530
batch: 10531
batch: 10532
batch: 10533
batch: 10534
batch: 10535
batch: 10536
batch: 10537
batch: 10538
batch: 10539
batch: 10540
batch: 10541
batch: 10542
batch: 10543
batch: 10544
batch: 10545
batch: 10546
batch: 10547
batch: 10548
batch: 10549
batch: 10550
batch: 10551
batch: 10552
batch: 10553
batch: 10554
batch: 10555
batch: 10556
batch: 10557
batch: 10558
batch: 10559
batch: 10560
batch: 10561
batch: 10562
batch: 10563
batch: 10564
batch: 10565
batch: 10566
batch: 10567
batch: 10568
batch: 10569
batch: 10570
batch: 10571
batch: 10572
batch: 10573
batch: 10574
batch: 10575
batch: 10576
batch: 10577
batch: 10578
batch: 10579